In [1]:
from tastytrade.logging import setup_logging
import logging
import asyncio
from tastytrade.sessions import Credentials
from tastytrade.sessions.requests import AsyncSessionHandler
from tastytrade.sessions.sockets import WebSocketManager
from decimal import Decimal
from datetime import datetime, timedelta
from tastytrade.markets.instruments import get_option_chains

import polars as pl

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

In [2]:
loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

In [ ]:
shutdown = asyncio.Event()

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)



def get_trade_day() -> str:
    trade_day = datetime.now()
    while trade_day.weekday() >= 5:
        trade_day += timedelta(days=1)

    return trade_day.strftime("%y%m%d")


def round_decimal(value, precision=5) -> float:
    target = str(precision)

    if not isinstance(value, Decimal):
        value = Decimal(value)

    if precision > 0:
        result = (value / Decimal(target)).quantize(
            Decimal("1"), rounding="ROUND_HALF_UP"
        ) * Decimal(target)
        return float(result)

    else:
        return float(round(value, abs(precision)))

In [4]:
yy_mm_dd = get_trade_day()

symbols = [
    "SPX",
    "NVDA",
    "BTC/USD:CXTALP",
    "BCH/USD:CXTALP",
    "ETH/USD:CXTALP",
    f".SPXW{yy_mm_dd}C5915",
    f".SPXW{yy_mm_dd}C5910",
    f".SPXW{yy_mm_dd}P5910",
    f".SPXW{yy_mm_dd}P5905",
]

## Test individual components

In [ ]:
# Start Rest API session
session = await AsyncSessionHandler.create(Credentials(env=ENV))

# Open WebSocket connection
dxlink = WebSocketManager(session)
await dxlink.open()
await dxlink.subscribe(symbols)

In [ ]:
loop.is_running()

In [ ]:
for task in asyncio.all_tasks():
    # if task.get_name().startswith("Task"):
    print(f"Task name: {task.get_name()}")
    print(f"Task coro: {task.get_coro()}")
    print(f"Task frame: {task.get_coro().cr_frame}")
    print(f"Task frame locals: {task.get_coro().cr_frame.f_locals}")
    print("---")

In [ ]:
dxlink.queue_manager.handlers["Trade"].processors["feed"].df

In [ ]:
dxlink.queue_manager.handlers["Greeks"].processors["feed"].df

In [ ]:
dxlink.queue_manager.handlers["Quote"].processors["feed"].df

In [ ]:
dxlink.queue_manager.handlers["Profile"].processors["feed"].df

In [ ]:
dxlink.queue_manager.handlers["Summary"].processors["feed"].df

In [ ]:
await asyncio.sleep(5)
await dxlink.close()

In [ ]:
dxlink.queue_manager.queues

In [ ]:
import sys

sys.exit()

In [11]:
symbol = "SPX"

response = session.session.get(session.base_url + "/option-chains/" + symbol)

In [ ]:
# Using your existing session
df = await get_option_chains(session, "SPX")
print(df)

In [5]:
from enum import Enum


class OptionType(Enum):
    CALL = "C"
    PUT = "P"


date = get_trade_day()
trade_date = f"20{date[:2]}-{date[2:4]}-{date[4:]}"

In [ ]:
(
    df.with_columns(pl.col("strike-price").cast(pl.Float32(), strict=False))
    .with_columns(
        pl.col("option-type").map_elements(lambda x: OptionType(x).name, return_dtype=str)
    )
    .filter((pl.col("expiration-date") == trade_date))
    .filter((pl.col("strike-price") >= 5700))
    .select(["strike-price", "option-type", "streamer-symbol"])
    .sort("strike-price")
    .head(10)
)

In [ ]:
vars(dxlink.queue_manager)

In [ ]:
from tastytrade.sessions.enumerations import Channels, EventTypes

Channels.Greeks
EventTypes.Greeks

In [ ]:
list(EventTypes.Greeks.value.model_fields.keys())

In [ ]:
list(EventTypes.Profile.value.model_fields.keys())

In [ ]:
[event for event in EventTypes]

In [ ]:
EventTypes["Greeks"]